<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction</a></span></li><li><span><a href="#Dataset-exploration" data-toc-modified-id="Dataset-exploration-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Dataset exploration</a></span><ul class="toc-item"><li><span><a href="#Content-of-the-files" data-toc-modified-id="Content-of-the-files-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Content of the files</a></span></li><li><span><a href="#Importing-the-data" data-toc-modified-id="Importing-the-data-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Importing the data</a></span></li></ul></li><li><span><a href="#Detecting-similar-books" data-toc-modified-id="Detecting-similar-books-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Detecting similar books</a></span><ul class="toc-item"><li><span><a href="#Using-titles-to-obtain-candidate-similars" data-toc-modified-id="Using-titles-to-obtain-candidate-similars-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Using titles to obtain candidate similars</a></span></li><li><span><a href="#Enriching-our-dataset-using-Amazon-product-Advertising-API" data-toc-modified-id="Enriching-our-dataset-using-Amazon-product-Advertising-API-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Enriching our dataset using Amazon product Advertising API</a></span></li><li><span><a href="#Using-author-list-to-refine-candidate-similars" data-toc-modified-id="Using-author-list-to-refine-candidate-similars-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Using author list to refine candidate similars</a></span></li><li><span><a href="#Using-titles-to-obtain-our-final-set-of-similar-books" data-toc-modified-id="Using-titles-to-obtain-our-final-set-of-similar-books-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Using titles to obtain our final set of similar books</a></span></li><li><span><a href="#Saving-the-data-using-pickle-for-further-analysis" data-toc-modified-id="Saving-the-data-using-pickle-for-further-analysis-3.5"><span class="toc-item-num">3.5&nbsp;&nbsp;</span>Saving the data using pickle for further analysis</a></span></li></ul></li><li><span><a href="#Correlation-analysis" data-toc-modified-id="Correlation-analysis-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Correlation analysis</a></span></li><li><span><a href="#Herding-Effect-analysis" data-toc-modified-id="Herding-Effect-analysis-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Herding Effect analysis</a></span></li></ul></div>

In [4]:
%matplotlib inline
# To reload external scripts automatically
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import sys
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
import os

# Importing external files
from scripts.similarities import *
from scripts.amazon_api_interaction import *
from scripts.analysis import *
from scripts.data_import import *
from scripts.utils_project import *


pd.options.mode.chained_assignment = None  # default='warn', Mutes warnings when copying a slice from a DataFrame.

In this notebook, we have tried to depict the full pipeline that we used for this project. It will not mention all the research that was done initially on a smaller dataset but those can be found in the ```research.ipynb```. Also note that most of the methods we use have been externalized to python files that can be found in ```scripts```.

# Introduction

_The goal of this project is to study the effect of the first review on an amazon product to subsequent reviews. This is also called the Herding effect._

Before we get started we wanted to explain our strategy. We have decided to focus on Amazon books as those make a great product to study the Herding effect as they are often available in different edition which do not modify the content.
1. The most challenging part was to **Identify similar books**. Indeed with a very large number of books it was complex to use pair wise comparison to do such task. Therefore we used Locality sensitive Hashing on the titles of the books (you can refer to the section for further explanations).
    * This method yields bins of candidate similar books that needs further refinements. While we first wanted to use the description of the book to find for example authors and other information, it turns out that most description are simply comments from journalists on the book and is therefore not that helpful. We decided to use **Amazon product advertising API** to obtain more metadata on the books that we found to be candidate similars.
    * Then **we used the authors of each book to refine candidate similars** where we tried to match the list of authors between two candidates by matching each author in a list to the author in the second book that had the least Levenstein distance. Once this matching was realized we look at the mean distance (the levenstein distance was transformed into a relative value to allow the comparison) and we consider valid two books whose mean relative levenstein distance is lower than 0.35. This phase allowed to delete books that had similar titles but not at all similar authors.
    * Finally we cleaned the titles from any unnecessary characters (parenthesis, colon, all lower case etc.) and decided to match two books if and only if the **symmetric difference of their normalized titles** is empty (which means that one of the set is entirely contained in the other).
2. Once we had those similar books, the real analysis can start. First of all we wanted to study the impact of reviews on the way user shop on the website (by using several metrics, trying for example to see the impact of average rating on the sales rank)
3. The very last step consisted in showing the Herding effect:
    * **Pre-analysis** of the obtained dataset was necessary in order to categorize the books into 6 categories (```HH,HM,HL,MM,ML,LL```) where ```H``` means that the first review of a book is High, ```M``` means that it is medium and ```L``` it is low. (you can refer to the section itself to see how we mapped those categories to the number of stars a book had)
    * We then plotted the average ratings for each groups depending on the fact that the book belong to the first or second letter of its category in order to see the **influence of the first review on the following**. Because of a quick convergence toward a similar value, we only analyzed the effect on the 5 first reviews.
    * As a final step we wanted to show the review average for each group on a **long term basis**. That is we wanted to see if on average the same book that had a first ```L``` rating has a lower review average in the long term than when it had a ```H``` first rating (comparing ```H``` in ```HL``` and ```L``` in ```HL```)

In [5]:
DATA_FOLDER = "../../Project-Data/"
META_FOLDER = DATA_FOLDER + "meta/"
REVIEWS_FOLDER = DATA_FOLDER + "reviews/"
CORE_FOLDER = DATA_FOLDER + "5_core/"
DUMP_FOLDER = DATA_FOLDER + "dump/"
CATEGORIES = ['Books','Movies_and_Tv','Electronics']
MAXCOUNT = -1
ANALYSIS_DATA_FOLDER = 'analysis_data/'

# Dataset exploration

## Content of the files

Because we have a lot of data we decided to first look at the content of the dataset in order to import into dataframes only the metadata that can be useful for our study. Note that we have three different types of file: **metadata**, **reviews** and **5-core** (which contains only reviews on products and reviewers that have at least 5 reviews).

The different columns of the **metadata** files are : 
* ```asin``` : the unique identifier of the object
* ```brand```
* ```categories``` : the categories of the object
* ```description``` : the description of the object
* ```imUrl```  : the link toward the images related to the object
* ```price```
* ```related``` : a list of objects that are related to this object
* ```salesRank``` 
* ```title```

The different columns of the **reviews** and **5-core** files are :
* ```asin``` : the unique identifier of the object
* ```helpful``` : a list of 2 integers [x,y], the helpfulness score is x/y votes
* ```overall``` : the rating of the object
* ```reviewText```
* ```reviewTime```
* ```reviewerID```
* ```reviewerName```
* ```summary ``` : the title of the review
* ```unixReviewTime``` : in Unix format

Therefore we keep only the column that are of interest for our task

In [6]:
meta_interesting_cols = ['asin', 'title', 'salesRank', 'description','imUrl']
review_interesting_cols = ['asin', 'overall', 'unixReviewTime']

## Importing the data

Working with large JSON would add a layer of complexity and therefore we imported the files to dataframes. A full pipeline has been made to do so. (NB: even-though the 5 core data was used for the exploratory phase we do not use it in the final study)

In [7]:
meta_books_path, review_books_path, core_book_path = get_paths(
    0, DATA_FOLDER, META_FOLDER, CORE_FOLDER, REVIEWS_FOLDER, CATEGORIES)
meta_books = import_interesting_cols(
    meta_books_path,
    DUMP_FOLDER,
    True,
    meta_interesting_cols,
    max_count=MAXCOUNT)
review_books = import_interesting_cols(
    review_books_path,
    DUMP_FOLDER,
    False,
    review_interesting_cols,
    max_count=MAXCOUNT)

Paths : 
	 meta = ../../Project-Data/meta/meta_Books.json
	 review = ../../Project-Data/reviews/reviews_Books.json
	 core_path = ../../Project-Data/5_core/Books.json
Retrieving from : ../../Project-Data/dump/meta_Books_asin_title_salesRank_description_imUrl_ALL
It took 00:00:05.051 to import the data.
Retrieving from : ../../Project-Data/dump/reviews_Books_asin_overall_unixReviewTime_ALL
It took 00:00:02.071 to import the data.


In [9]:
print("The meta_books dataframe has {} books and {} attributes".format(meta_books.shape[0],meta_books.shape[1]))
meta_books.head(1)

The meta_books dataframe has 2370585 books and 21 attributes


,asin,description,imUrl,"salesRank_Arts,_Crafts_&_Sewing",salesRank_Books,salesRank_Cell_Phones_&_Accessories,salesRank_Clothing,salesRank_Electronics,salesRank_Health_&_Personal_Care,salesRank_Home_&_Kitchen,...,salesRank_Jewelry,salesRank_Kitchen_&_Dining,salesRank_Movies_&_TV,salesRank_Music,salesRank_Musical_Instruments,salesRank_Office_Products,salesRank_Shoes,salesRank_Sports_&_Outdoors,salesRank_Toys_&_Games,title
0,0001048791,NaN,http://ecx.images-amazon.com/images/I/51MKP0T4...,NaN,6334800.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"The Crucible: Performed by Stuart Pankin, Jero..."


In [10]:
print("The review_books dataframe has {} reviews and {} attributes".format(review_books.shape[0],review_books.shape[1]))
review_books.head(1)

The review_books dataframe has 22507155 books and 3 attributes


,asin,overall,unixReviewTime
0,0000000116,4.0,2002-04-27


# Detecting similar books

We will now start to detect similar books. We have in previous exploratory discarded reviewers with less than 5 reviews and books with less than 5 reviews. While this latter does make sense for the Herding Effect analysis, the restriction on reviewers doesn't make much sense. We will therefore still keep the restriction on books of having at least 5 reviews.

In [11]:
print("In the full dataset we have {} books and  {} reviews".format(meta_books.shape[0],review_books.shape[0]))
books_reviews_count = review_books.groupby('asin').count().overall
asin_enough_reviews = list(books_reviews_count[books_reviews_count > 5].index)
print("We found {} books with enough reviews.".format(len(asin_enough_reviews)))

In the full dataset we have 2370585 books and  22507155 reviews
We found 625682 books with enough reviews.


Now that we have obtained the identifiers of the books with enough reviews we will keep only those in the ```meta_books``` dataframe.

In [12]:
meta_books_reduced = meta_books[meta_books['asin'].isin(asin_enough_reviews)]
print("Now the books dataset is composed of {} books".format(meta_books_reduced.shape[0]))

Now the books dataset is composed of 625682 books


Since we are working on large datasets we took the habit to serialize our intermediate work to reduce the running time of the code. We will serialize those in the ```../../Project-Data/dump_full/``` folder (you can refer to the ```README``` to observe the file architecture of the project). 

In [13]:
DUMP_FOLDER_FULL = DATA_FOLDER + 'dump_full/'

## Using titles to obtain candidate similars

Using LSH (local sensitivity hashing) we will now try to find bins of candidate similars using the tiltes of the amazon products. Therefore we will keep in our dataframe only the books with an existing title and only the columns that can later be of interest.

In [15]:
meta_books_reduced  = meta_books_reduced [meta_books_reduced ['title'].notnull()]
meta_books_reduced = meta_books_reduced[['asin','title','description','imUrl']]
meta_books_reduced.head()

,asin,title,description,imUrl
10,0000913154,The Way Things Work: An Illustrated Encycloped...,NaN,http://ecx.images-amazon.com/images/I/7113akhD...
13,0001055178,Master Georgie,Beryl Bainbridge seems drawn to disaster. Firs...,http://ecx.images-amazon.com/images/I/51ZSC6TK...
25,0001018043,The Enchanted Horse,"Grade 3-6-In this fast-paced fantasy, a neglec...",NaN
34,0001473727,"The Greatest Book on ""Dispensational Truth"" in...",NaN,http://ecx.images-amazon.com/images/I/512M299K...
37,0000230022,The Simple Truths of Service: Inspired by John...,NaN,http://ecx.images-amazon.com/images/I/218uMkP0...


In [16]:
dump_path = DUMP_FOLDER_FULL
get_candidate_full = candidate_duplicates(meta_books_reduced,dump_path,['title'])

Retrieving from : ../../Project-Data/dump_full/candidate_dup_title_2_100_5_4
Found 15449 bins of possible duplicates.
With 36066 different books


*NB: the hashing used involves random number and therefore different run of the method ```candidate_duplicates``` might result in slightly different bins. This is one of the reason why we also serialize the candidates in order to always have the same data to deduct conclusion in further work*

## Enriching our dataset using Amazon product Advertising API

Because amazon API has strong limitations on the number of requests per account and the number of queries per seconds, we try to use it as efficiently as possible:
* we decided to limit our QPS (query per second) to 0.9 as the limit is 1 QPS. 
* whenever the API throttles our queries we wait a random amount of time before retrying (following a random variable with exponential distribution of parameter 0.1, as advised by Amazon).
* each query retrieves 10 books' details at a time.

This will allow us to get many more details of our books : ```authors, publisher, ISBN, sales_rank_updated, binding, edition, release_date```. We might not need all of them but because of the restriction of the API we decided to obtain as much as possible for each book should we decide to use it later on.

In [17]:
amazon_access_file_path = DUMP_FOLDER + "amazon_access_epfl_new"
book_only_candidates,failed = fill_in_with_details(meta_books_reduced,amazon_access_file_path,get_candidate_full,DUMP_FOLDER_FULL)
book_only_candidates.head(2)

Retrieving from : 
	../../Project-Data/dump_full/book_only_candidates_with_details
	../../Project-Data/dump_full/api_failed 
It took 00:00:00.181 to get the data.


,asin,title,description,imUrl,authors,publisher,ISBN,sales_rank_updated,binding,edition,release_date
0,0001473727,"The Greatest Book on ""Dispensational Truth"" in...",NaN,http://ecx.images-amazon.com/images/I/512M299K...,[C. Larkin],Titles distributed by Christian Art Distributors,0001473727,837246,Relié,None,None
1,0001472933,The Book of Daniel,NaN,http://ecx.images-amazon.com/images/I/414KH1FP...,[],Rev Clarence Larkin Estate,0001472933,1569511,Relié,None,None


Some book ASIN might not have been successfully retrieved for numerous reasons, one of which might be that the object isn't sold anymore. We saved such failed ASIN in the ```failed``` list.

In [19]:
print("book_only_candidates.shape = ",book_only_candidates.shape)
print("{} have failed".format(len(failed)))

book_only_candidates.shape =  (36066, 11)
550 have failed


## Using author list to refine candidate similars

Because the matching will now use authors of each books, we will eliminate from our dataset the books for which no authors was found (in the case where the API couldn't retrieve the details of the book using the ASIN) but also in the case where it is an empty list (the API call was successful but no author was returned.

In [20]:
book_only_candidates = book_only_candidates[book_only_candidates['authors'].notnull()]
book_only_candidates = book_only_candidates[book_only_candidates['authors'].apply(lambda x:len(x)>0)]
print("This leaves us with {} books to work with".format(book_only_candidates.shape[0]))

This leaves us with 33337 books to work with


Now we also need to keep in our dictionnary ```get_candidate_full```, obtained from LSH, only the books that have an author list that allow us to match them.

In [21]:
compliant_asins = set(book_only_candidates['asin'])
similar_sets = [set(get_candidate_full[k]+[k]) for k in get_candidate_full.keys()]
similar_sets_compliant=[]
for s in similar_sets:
    inter = s & compliant_asins
    if(len(inter)>1):
        similar_sets_compliant.append(inter)
print("We now have {} bins of similar books to work with".format(len(similar_sets_compliant)))

# And we put it back into a dictionnary
similars = [sorted(list(s)) for s in similar_sets_compliant]
get_candidate = {elements[0]:elements[1:] for elements in similars}
book_only_candidates = book_only_candidates.set_index('asin')

We now have 13913 bins of similar books to work with


We saw examples where the authors list between two books were very similar but names were written differently, therefore we need to check that some typos are not in the names of authors : (e.g. ```{'Ian MacNeill', 'SportMed BC'}{'Ian MacNeil'}``` where MacNeil is written with two 'l' in the first book but only one in the second one). Therefore we will first clean the names (remove accents, nomalize it to lower case) and then try to match then using the Levenstein distance, we match two author list if the mean difference between authors is less that 0.35 (NB: this is a relative distance in order to be able to compare title with different length). 

See the function ```check_name_similarity``` in the ```utils.py``` files for more details on how this is done.

In [22]:
book_only_candidates['authors'] = book_only_candidates['authors'].apply(clean_name_in_dataframe)
similars = get_similar_authors(book_only_candidates,get_candidate,0.35)
print("We have found {} bins of candidate similars".format(len(similars.keys())))

We have found 5987 bins of candidate similars


We can see that we have some unwanted behaviour of the author matching function. Indeed because we want to avoid at all cost false positives we have some matching that could trivially be done by humans : 'sir arthur conan doyle' and 'a conan doyle' are considered but should be considered similar. We decided to leave the task of refining the matching to a later study and possibly to a reader that would like to extend our work.

## Using titles to obtain our final set of similar books

Now we might still have problems with books that have highly similar titles and authors but aren't identical. This could be explained by :
* Different Tomes : same title but different number
* educational books that have identical titles except for what they teach
* simply typos or very small differences 
* many other examples 

Example of books that are not similar:
```
Adobe Dreamweaver CS3 Classroom in a Book['adobe creative team']
Adobe Dreamweaver CS4 Classroom in a Book['adobe creative team']
Adobe Dreamweaver CS5 Classroom in a Book['adobe creative team']
Adobe Dreamweaver CS6 Classroom in a Book['adobe creative team']
----------------------------------------------------------------
Promethea, Book 3['alan moore']
Promethea, Book 5['alan moore']
Promethea, Book 1['alan moore', 'j. h. williams', 'mick gray']

```

Example of books that should be considered similar:
``` 
'Light Science and Magic: An Introduction to Photographic Lighting'['fil hunter', 'paul fuqua', 'steven biver']
'Light: Science and Magic: An Introduction to Photographic Lighting'['fil hunter', 'steven biver', 'paul fuqua']
```

We will therefore also need to see if the title are similar enough. **We want to avoid false positive as much as possible. We are sure that we could go even further with matching names but prefer to be conservative here**. Therefore we only try to normalize as much as possible the titles and see if then the titles taken as sets have an empty symmetric difference.

In [23]:
book_only_candidates['title'] = book_only_candidates['title'].apply(clean_title)
very_similars = get_similar_titles(book_only_candidates,similars)
print("We have found {} bins of very similar books.".format(len(very_similars.keys())))
very_similar_ASIN = list(very_similars.keys())
very_similar_ASIN += [item for sublist in list(very_similars.values()) for item in sublist]
print("We have {} asins for products that have at least one similar product.".format(len(very_similar_ASIN)))

We have found 3050 bins of very similar books.
We have 6410 asins for products that have at least one similar product.


Using the ```observational_print``` argument of ```get_similar_titles``` one can observe how our function is quite conservative and why:

An example where the titles were not matched, providing there difference:
```
'college mathematics for business economics life sciences and social sciences 12th edition barnett'
'finite mathematics for business economics life sciences and social sciences 12th edition barnett'
	 --->'college','finite'
```
and another one
```
'mastering the nikon d700'
'mastering the nikon d600'
	 --->'d700','d600
```

We can see what titles are going to be matched : 
*What is interesting here is that the last title contains a repetition of the title inside parenthesis, after the cleaning, we will have removed the parenthesis and will check if the symmetric difference in terms of set of words is empty.*
```
"Jeff Herman's Guide to Book Publishers, Editors & Literary Agents: Who They Are! What They Want! and How to Win Them Over!",
"Jeff Herman's Guide to Book Publishers, Editors and Literary Agents : Who They Are! What They Want! How to Win Them Over!",
"Jeff Herman's Guide to Book Publishers, Editors, and Literary Agents: Who They Are! What They Want! How to Win Them Over! (Jeff Herman's Guide to Book Publishers, Editors, & Literary Agents)"
```
*or this second example where a ':' has been replace with a ';'*
```
'Hawaii the Big Island Revealed: The Ultimate Guidebook',
'Hawaii The Big Island Revealed; The Ultimate Guidebook',
```


While we know that we might be missing a lot of titles, we much rather have less title and 0 false positives than to risk having false positives.

## Saving the data using pickle for further analysis
Now that we have obtained the dataset that we wish to use to conduct our analysis it is time to save it locally to avoid long and repetitive operations while we do the analysis.

In [26]:
ANALYSIS_DATA_FOLDER = 'analysis_data/'
books_df_path = ANALYSIS_DATA_FOLDER + 'books_full'
review_books_df_path = ANALYSIS_DATA_FOLDER + 'review_books_full'
very_similars_path = ANALYSIS_DATA_FOLDER + 'very_similars_full'

paths = [books_df_path,review_books_df_path,very_similars_path]

In [39]:
retrieved = []
# Get the file from the dump
if(all(os.path.isfile(p) for p in paths)):
    # We check if the files already exist
    print("Retrieving from :")
    for i,p in enumerate(paths):
        print("\t{}".format(p))
        if(i<2):
            df = pd.read_pickle(p)
            retrieved.append(df)
        else:
            with open(p, 'rb') as handle:
                very_similars = pickle.load(handle)
                retrieved.append(very_similars)
else:
    # Otherwise we create them
    book_only_candidates.loc[very_similar_ASIN].to_pickle(ANALYSIS_DATA_FOLDER + 'books_full')
    review_books.query('asin in @very_similar_ASIN').to_pickle(ANALYSIS_DATA_FOLDER + 'review_books_full')
    with open(ANALYSIS_DATA_FOLDER + 'very_similars_full', 'wb') as handle:
        pickle.dump(very_similars, handle, protocol=pickle.HIGHEST_PROTOCOL)

books_df,review_books_df,very_similars=retrieved
print("We have retrieved:\n\tbooks_df with shape: {}\n\treview_books_df with shape: {}\n\tvery_similars with length: {}".format(books_df.shape,review_books_df.shape,len(very_similars)))

Retrieving from :
	analysis_data/books_full
	analysis_data/review_books_full
	analysis_data/very_similars_full
We have retrieved:
	books_df with shape: (6410, 10)
	review_books_df with shape: (361389, 3)
	very_similars with length: 3050


# Correlation analysis

# Herding Effect analysis